In [1]:
{-# LANGUAGE DataKinds, FlexibleContexts, QuasiQuotes, TemplateHaskell #-}

In [2]:
import Frames
import Graphics.Rendering.Chart
import Data.Foldable

The goal is to add a new type to the "universe of types". In this case we want to allow Haskell to infer that a column is a Gender column by looking for the strings "male" and "female"
Code grabbed from the https://github.com/acowley/Frames/blob/master/demo/TutorialUsers.hs

In [17]:
{-# LANGUAGE DataKinds, DeriveDataTypeable, TypeFamilies, TypeOperators, OverloadedStrings #-}
module TitanicTypes where
import Control.Monad (mzero)
import qualified Data.Char as C
import Data.Readable (Readable(fromText))
import qualified Data.Text as T
import Data.Typeable
import qualified Data.Vector as V
import Frames.InCore (VectorFor)
import Frames.ColumnTypeable (Parseable)

import Frames

-- Making GenderT
data GenderT = Male | Female deriving (Enum,Eq,Ord,Show,Typeable)

type instance VectorFor GenderT = V.Vector

instance Readable GenderT where
  fromText "male" = return Male
  fromText "female" = return Female
  fromText _ = mzero
  
-- Very important to make GenderT an instance of Parsable!!!
instance Parseable GenderT where

-- Making Survived (basically the same thing!)

data SurvivedT = Survived | Died deriving (Enum,Eq,Ord,Show,Typeable)

type instance VectorFor SurvivedT = V.Vector

instance Readable SurvivedT where
  fromText "1" = return Survived
  fromText "0" = return Died
  fromText _ = mzero
        
instance Parseable SurvivedT where

-- Add a "class" type
data ClassT = First | Second | Third deriving (Enum, Eq, Ord, Show, Typeable)

type instance VectorFor ClassT = V.Vector

instance Readable ClassT where
  fromText "1" = return First
  fromText "2" = return Second
  fromText "3" = return Third
  fromText _ = mzero
  
instance Parseable ClassT where

-- Typing the Embarked column.

data EmbarkedT = Queenstown | Cherbourg | Southampton deriving (Enum,Eq,Ord,Show,Typeable)

type instance VectorFor EmbarkedT = V.Vector

instance Readable EmbarkedT where
  fromText "Q" = return Queenstown
  fromText "C" = return Cherbourg
  fromText "S" = return Southampton
  fromText _ = mzero
  
instance Parseable EmbarkedT where


-- Add to the new types. 
type MyColumns = EmbarkedT ': ClassT ': SurvivedT ': GenderT ': CommonColumns
-- type MyColumns = EmbarkedT ':  GenderT ': CommonColumns


Following along with the Frames tutorial, we apparently need to use the following Template haskell

In [15]:
import Control.Applicative
import qualified Control.Foldl as L
import qualified Data.Foldable as F
import Data.Proxy (Proxy(..))
import Frames
import Frames.CSV (readTableOpt, rowGen, RowGen(..))
import Frames.CSV (colQ)
import TitanicTypes


tableTypes' rowGen { rowTypeName = "CustRow"
                   , tablePrefix = "u2"
                   , columnUniverse = $(colQ ''MyColumns) }
            "data/train.csv"

No instance for (Parseable SurvivedT) arising from a use of tableTypes'
In the expression: tableTypes' (rowGen {rowTypeName = "CustRow", tablePrefix = "u2", columnUniverse = Proxy :: Proxy (ColumnUniverse MyColumns)}) "data/train.csv"
In an equation for `it': it = tableTypes' (rowGen {rowTypeName = "CustRow", tablePrefix = "u2", columnUniverse = Proxy :: Proxy (ColumnUniverse MyColumns)}) "data/train.csv"

In [16]:
:i CustRow

Not in scope: `CustRow'

Now the Type `CustRow` has correctly parsed the "Sex", "Survived", "Embarked" and "Class". However, "SubSp" and "Parch" have now also become a `SurvivedT` type!

This is clearly an error as Parch (number of parents) and SubSp (number of children) can be more than just 0 or 1. 

We can use the function `tableTypes` from the `Frames` library to generate a type `Row` for our CSV file, after that we can make a loader function that will load a `Frame` from the file.

In [6]:

loadRows :: IO (Frame CustRow)
loadRows = inCoreAoS $ readTable "data/train.csv"

Not in scope: type constructor or class `CustRow'

In [7]:
loadRows

Not in scope: `loadRows'

We can inspect the **row** type:

In [8]:
:i CustRow

Not in scope: `CustRow'

It gives us the following:
```hs
type Row =
    Record
        '["PassengerId" :-> Int
         , "Survived"   :-> SurvivedT
         , "Pclass"     :-> Int
         , "Name"       :-> Text
         , "Sex"        :-> GenderT
         , "Age"        :-> Int
         , "SibSp"      :-> Int
         , "Parch"      :-> Int
         , "Embarked"   :-> Int
         ]
```

In [9]:
rows <- loadRows

Not in scope: `loadRows'

In [10]:
rec1 = head $ toList rows
rec1

Not in scope: `rows'

We can see that this visualization style is not really comfortable, as it is just printing the `Text` representation of the record. We could try to render this as an HTML table:

In [11]:
import Graphics.Rendering.Chart.Easy
import IHaskell.Display
import Data.Vinyl.Core
import Data.Vinyl



mapM_ print $ showFields rec1

Not in scope: `rec1'
Perhaps you meant one of these: `IHaskellTH.recP' (imported from Language.Haskell.TH), `IHaskellTH.recC' (imported from Language.Haskell.TH)

In [12]:
import Language.Haskell.TH

import Data.Proxy
import qualified Data.Text as T
import Data.Typeable (Typeable, showsTypeRep, typeRep)
import Data.Vinyl
import Data.Vinyl.Functor
import Frames.CoRec
import Frames.ColumnTypeable
import Frames.RecF (reifyDict)
import Frames.TypeLevel (LAll)
import Data.Typeable (TypeRep)
import Data.Maybe (fromMaybe)
import Data.Readable (Readable(fromText))


In [13]:
:i Readable